In [1]:
# Import modules
import ast
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter

from scipy.spatial.distance import cdist

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances

from tqdm import tqdm


In [2]:
# Import the data
followers_df = pd.read_csv('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_management/db_soundcharts/editorial_tracks_followers_23-24.csv')
main_df = pd.read_csv('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_management/editorial_playlists_23-24_final.csv')
display(followers_df)


,playlist_id,playlist_followers,playlist_followers_date
0,11e84493-3e36-359e-a69a-a0369fe50396,2191622,2023-10-01T00:00:00+00:00
1,11e84493-3e36-359e-a69a-a0369fe50396,2191622,2023-10-01T00:00:00+00:00
2,11e84493-3e36-359e-a69a-a0369fe50396,2191622,2023-10-01T00:00:00+00:00
3,11e84493-3e36-359e-a69a-a0369fe50396,2191622,2023-10-01T00:00:00+00:00
4,11e84493-3e36-359e-a69a-a0369fe50396,2191622,2023-10-01T00:00:00+00:00
...,...,...,...
3950,11e84493-3db1-6b5c-8640-a0369fe50396,2478,2024-05-26T00:00:00+00:00
3951,11e84493-3db1-6b5c-8640-a0369fe50396,2478,2024-05-26T00:00:00+00:00
3952,11e84493-3db1-6b5c-8640-a0369fe50396,2478,2024-05-26T00:00:00+00:00
3953,11e84493-3db1-6b5c-8640-a0369fe50396,2478,2024-05-26T00:00:00+00:00


In [4]:
# Collapse to weekly level playlists data
# Convert date column to datetime
followers_df['playlist_followers_date'] = pd.to_datetime(followers_df['playlist_followers_date'])

# Extract year and week number
followers_df['year'] = followers_df['playlist_followers_date'].dt.isocalendar().year
followers_df['week'] = followers_df['playlist_followers_date'].dt.isocalendar().week
followers_df['weekday'] = followers_df['playlist_followers_date'].dt.weekday  # Monday=0, ..., Sunday=6

# Sort data to ensure selection priority
followers_df.sort_values(by=['playlist_id', 'year', 'week', 'weekday'], ascending=[True, True, True, True], inplace=True)

# Keep only one entry per playlist_id per week, prioritizing Friday (4)
df_weekly = followers_df.groupby(['playlist_id', 'year', 'week']).apply(lambda x: x[x['weekday'] == 4].iloc[:1] if (x['weekday'] == 4).any() else x.iloc[:1])

# Reset index and drop unnecessary columns
df_weekly = df_weekly.droplevel(0).reset_index(drop=True)
df_weekly.drop(columns=['year', 'week', 'weekday'], inplace=True)

# Display result
display(df_weekly)


,playlist_id,playlist_followers,playlist_followers_date
0,11e84493-3bb2-2850-8a3a-a0369fe50396,217419,2023-10-01 00:00:00+00:00
1,11e84493-3bb2-2850-8a3a-a0369fe50396,217269,2023-10-14 00:00:00+00:00
2,11e84493-3bb2-2850-8a3a-a0369fe50396,217373,2023-10-21 00:00:00+00:00
3,11e84493-3bb2-2850-8a3a-a0369fe50396,217499,2023-10-28 00:00:00+00:00
4,11e84493-3bb2-2850-8a3a-a0369fe50396,217743,2023-11-04 00:00:00+00:00
...,...,...,...
682,dd7887d5-516d-11e9-8475-246e96c3621c,152698,2024-08-31 00:00:00+00:00
683,dd7887d5-516d-11e9-8475-246e96c3621c,152765,2024-09-07 00:00:00+00:00
684,dd7887d5-516d-11e9-8475-246e96c3621c,152989,2024-09-14 00:00:00+00:00
685,dd7887d5-516d-11e9-8475-246e96c3621c,153200,2024-09-21 00:00:00+00:00
